In [8]:
import logging
import uuid
from textwrap import dedent

from dotenv import load_dotenv

In [10]:
from typing import List, Annotated

from langchain_core.messages import HumanMessage, SystemMessage, AnyMessage
from langchain_core.pydantic_v1 import BaseModel
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import END, MessageGraph, add_messages

In [11]:
load_dotenv()
logger = logging.getLogger(__name__)
console = logging.StreamHandler()
logger.addHandler(console)
logger.setLevel(logging.DEBUG)

In [12]:
llm = ChatOpenAI(temperature=0)

In [13]:
template = dedent(
    """
Your job is to get information from a user about what type of prompt template they want to create.

You should get the following information from them:

- What the objective of the prompt is
- What variables will be passed into the prompt template
- Any constraints for what the output should NOT do
- Any requirements that the output MUST adhere to

If you are not able to discern this info, ask them to clarify! Do not attempt to wildly guess.

After you are able to discern all the information, call the relevant tool
to generate the prompt template and return it to the user.
"""
)

In [15]:
def get_messages_info(
    messages: Annotated[List[AnyMessage], "messages"]
) -> List[AnyMessage]:
    logger.debug("get_messages_info: %s", messages)
    return [SystemMessage(content=template)] + messages


class PromptInstructions(BaseModel):
    """Instructions on how to prompt the LLM."""

    objective: str
    variables: List[str]
    constraints: List[str]
    requirements: List[str]

In [16]:
llm_with_tool = llm.bind_tools([PromptInstructions])

chain = get_messages_info | llm_with_tool
chain

RunnableLambda(get_messages_info)
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x12064ab50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x121466010>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'PromptInstructions', 'description': 'Instructions on how to prompt the LLM.', 'parameters': {'type': 'object', 'properties': {'objective': {'type': 'string'}, 'variables': {'type': 'array', 'items': {'type': 'string'}}, 'constraints': {'type': 'array', 'items': {'type': 'string'}}, 'requirements': {'type': 'array', 'items': {'type': 'string'}}}, 'required': ['objective', 'variables', 'constraints', 'requirements']}}}]})

In [17]:
# Helper function for determining if tool was called
def _is_tool_call(msg):
    logger.debug("Checking if tool was called: %s", msg)
    return hasattr(msg, "additional_kwargs") and "tool_calls" in msg.additional_kwargs

In [18]:
# New system prompt
prompt_system = dedent(
    """\
Based on the following requirements, write a good prompt template:

{reqs}
"""
)


# Function to get the messages for the prompt
# Will only get messages AFTER the tool call
def get_prompt_messages(messages):
    tool_call = None
    other_msgs = []
    for m in messages:
        if _is_tool_call(m):
            tool_call = m.additional_kwargs["tool_calls"][0]["function"]["arguments"]
        elif tool_call is not None:
            other_msgs.append(m)
    return [SystemMessage(content=prompt_system.format(reqs=tool_call))] + other_msgs

In [19]:
prompt_gen_chain = get_prompt_messages | llm

In [32]:
print(prompt_gen_chain)

first=RunnableLambda(get_prompt_messages) last=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x12064ab50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x121466010>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')


In [22]:
def get_state(messages):
    if _is_tool_call(messages[-1]):
        return "prompt"
    elif not isinstance(messages[-1], HumanMessage):
        return END
    for m in messages:
        if _is_tool_call(m):
            return "prompt"
    return "info"

In [37]:
memory: SqliteSaver = SqliteSaver.from_conn_string(":memory:")
nodes = {k: k for k in ["info", "prompt", END]}
workflow = MessageGraph()
print(dir(memory))
workflow.add_node("info", chain)
workflow.add_node("prompt", prompt_gen_chain)
workflow.add_conditional_edges("info", get_state, nodes)
workflow.add_conditional_edges("prompt", get_state, nodes)
workflow.set_entry_point("info")
graph = workflow.compile(checkpointer=memory)

['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', 'aget', 'aget_tuple', 'alist', 'aput', 'asearch', 'config_specs', 'conn', 'cursor', 'from_conn_string', 'get', 'get_tuple', 'is_setup', 'list', 'lock', 'put', 'search', 'serde', 'setup']


In [31]:
config = {"configurable": {"thread_id": str(uuid.uuid4())}}
while True:
    user = input("User (q/Q to quit): ")
    if user in {"q", "Q"}:
        print("AI: Byebye")
        break
    for output in graph.stream([HumanMessage(content=user)], config=config):
        if "__end__" in output:
            continue
        # stream() yields dictionaries with output keyed by node name
        for key, value in output.items():
            print(f"Output from node '{key}':")
            print("---")
            print(value)
        print("\n---\n")

get_messages_info: [HumanMessage(content='Hello', id='4465a8b5-70b5-4f2f-881d-599c7478f888')]
get_messages_info: [HumanMessage(content='Hello', id='4465a8b5-70b5-4f2f-881d-599c7478f888')]
Checking if tool was called: content='Hello! How can I assist you today?' response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 181, 'total_tokens': 191}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-9dd2e7bf-3a99-4b7f-84d7-bccdb08fe74a-0'
Checking if tool was called: content='Hello! How can I assist you today?' response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 181, 'total_tokens': 191}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-9dd2e7bf-3a99-4b7f-84d7-bccdb08fe74a-0'


Output from node 'info':
---
content='Hello! How can I assist you today?' response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 181, 'total_tokens': 191}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-9dd2e7bf-3a99-4b7f-84d7-bccdb08fe74a-0'

---



get_messages_info: [HumanMessage(content='Hello', id='4465a8b5-70b5-4f2f-881d-599c7478f888'), AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 10, 'prompt_tokens': 181, 'total_tokens': 191}}, id='run-9dd2e7bf-3a99-4b7f-84d7-bccdb08fe74a-0'), HumanMessage(content='I would like a prompt that chooses a color and the varialble is the color', id='fcb89bbb-d53c-41be-8ba9-f654e97a4a9f')]
get_messages_info: [HumanMessage(content='Hello', id='4465a8b5-70b5-4f2f-881d-599c7478f888'), AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 10, 'prompt_tokens': 181, 'total_tokens': 191}}, id='run-9dd2e7bf-3a99-4b7f-84d7-bccdb08fe74a-0'), HumanMessage(content='I would like a prompt 

Output from node 'info':
---
content='Great! Could you please provide more information about the objective of the prompt, any constraints for what the output should NOT do, and any requirements that the output MUST adhere to? This will help me create a prompt template that meets your needs.' response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 215, 'total_tokens': 264}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-1fc84770-166a-4295-b595-e14b33604dac-0'

---



get_messages_info: [HumanMessage(content='Hello', id='4465a8b5-70b5-4f2f-881d-599c7478f888'), AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 10, 'prompt_tokens': 181, 'total_tokens': 191}}, id='run-9dd2e7bf-3a99-4b7f-84d7-bccdb08fe74a-0'), HumanMessage(content='I would like a prompt that chooses a color and the varialble is the color', id='fcb89bbb-d53c-41be-8ba9-f654e97a4a9f'), AIMessage(content='Great! Could you please provide more information about the objective of the prompt, any constraints for what the output should NOT do, and any requirements that the output MUST adhere to? This will help me create a prompt template that meets your needs.', response_metadata={'finish_reason': 'stop', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 49, 'prompt_tokens

Output from node 'info':
---
content='' additional_kwargs={'tool_calls': [{'id': 'call_xtuiZSporCUumEJ1U27CJRA9', 'function': {'arguments': '{"objective":"Choose your favorite color and determine its opposite color.","variables":["color"],"constraints":[],"requirements":[]}', 'name': 'PromptInstructions'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 304, 'total_tokens': 337}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-335e3a91-c47f-445b-82e5-55f2ef372468-0' tool_calls=[{'name': 'PromptInstructions', 'args': {'objective': 'Choose your favorite color and determine its opposite color.', 'variables': ['color'], 'constraints': [], 'requirements': []}, 'id': 'call_xtuiZSporCUumEJ1U27CJRA9'}]

---



Checking if tool was called: content='What is your favorite color? Determine its opposite color.' response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 43, 'total_tokens': 54}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-1fa21b51-a1ce-4f45-92e5-1fa99fd31aea-0'
Checking if tool was called: content='What is your favorite color? Determine its opposite color.' response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 43, 'total_tokens': 54}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-1fa21b51-a1ce-4f45-92e5-1fa99fd31aea-0'


Output from node 'prompt':
---
content='What is your favorite color? Determine its opposite color.' response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 43, 'total_tokens': 54}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-1fa21b51-a1ce-4f45-92e5-1fa99fd31aea-0'

---

AI: Byebye


In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))